# Deep Q-Network (DQN)
---
Implemented a DQN agent with OpenAI Gym's LunarLander-v2 environment.

### 1. Import the Necessary Packages

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt

from dqn_agent import Agent
%matplotlib inline

### 2. Instantiate the Environment and Agent

Initialize the environment in the code cell below.

In [2]:
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

State shape:  (8,)
Number of actions:  4


### 3. Train the Agent with DQN

Training the agent from scratch.
- Defined a neural network architecture in `model.py` that maps states to action values.
- Defined DQN `Agent` in `dqn_agent.py`.  It samples a batch of experience tuples and uses the local and target Q-networks to compute the loss. Implemented "Experience Replay" and "Fixed Q-targets" concepts from DQN paper in the Agent.

In [3]:
def dqn(env, agent, n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            # get action based on the state.
            action = agent.act(state, eps)
            # get reward, next state, and done
            next_state, reward, done, _ = env.step(action)
            # collects experiences (or [S, A, R, S'] tuples) and uses fixed Q-targets
            # to learn (or train) the optimal action-value function.
            agent.step(state, action, reward, next_state, done)

            # reset state to next state
            state = next_state
            score += reward
            if done:
                break
        
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.q_network_local.state_dict(), 'checkpoint.pth')
            break
    return scores


In [4]:
agent = Agent(
    state_size=env.observation_space.shape[0], 
    action_size=env.action_space.n, 
    seed=42,
)

scores = dqn(env, agent)

Episode 100	Average Score: -162.49
Episode 200	Average Score: -119.81
Episode 300	Average Score: -10.818
Episode 400	Average Score: 53.948
Episode 500	Average Score: 156.93
Episode 600	Average Score: 171.09
Episode 700	Average Score: 183.35
Episode 800	Average Score: 185.28
Episode 900	Average Score: 185.40
Episode 938	Average Score: 200.20
Environment solved in 838 episodes!	Average Score: 200.20


AttributeError: 'Agent' object has no attribute 'qnetwork_local'

In [5]:
torch.save(agent.q_network_local.state_dict(), 'checkpoint.pth')

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Watch a Smart Agent!

In the next code cell, you will load the trained weights from file to watch a smart agent!

In [3]:
agent = Agent(
    state_size=env.observation_space.shape[0], 
    action_size=env.action_space.n, 
    seed=42,
)
# load the weights from file
agent.q_network_local.load_state_dict(torch.load('checkpoint.pth'))

for i in range(10):
    state = env.reset()
    for j in range(200):
        action = agent.act(state)
        env.render()
        state, reward, done, _ = env.step(action)
        if done:
            break 
            
env.close()